# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 15 2022 1:47PM,234465,12,TWL-4905,Twinlab Consolidated Corporation,Released
1,Apr 15 2022 1:38PM,234463,10,0085602617,ISDIN Corporation,Executing
2,Apr 15 2022 1:38PM,234463,10,0085602618,ISDIN Corporation,Executing
3,Apr 15 2022 1:38PM,234463,10,0085602611,ISDIN Corporation,Executing
4,Apr 15 2022 1:38PM,234463,10,0085602604,ISDIN Corporation,Executing
5,Apr 15 2022 1:38PM,234463,10,0085602612,ISDIN Corporation,Executing
6,Apr 15 2022 1:38PM,234463,10,0085602613,ISDIN Corporation,Executing
7,Apr 15 2022 1:38PM,234463,10,0085602615,ISDIN Corporation,Executing
8,Apr 15 2022 1:38PM,234463,10,0085602614,ISDIN Corporation,Executing
9,Apr 15 2022 1:38PM,234463,10,0085602616,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,234461,Released,16
52,234462,Released,1
53,234463,Executing,36
54,234464,Executing,1
55,234465,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
234461,NaN,NaN,16.0
234462,NaN,NaN,1.0
234463,NaN,36.0,NaN
234464,NaN,1.0,NaN
234465,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
234309,0.0,0.0,1.0
234310,0.0,0.0,6.0
234343,0.0,0.0,1.0
234344,0.0,0.0,24.0
234347,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
234309,0,0,1
234310,0,0,6
234343,0,0,1
234344,0,0,24
234347,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,234309,0,0,1
1,234310,0,0,6
2,234343,0,0,1
3,234344,0,0,24
4,234347,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,234309,,,1
1,234310,,,6
2,234343,,,1
3,234344,,,24
4,234347,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 15 2022 1:47PM,234465,12,Twinlab Consolidated Corporation
1,Apr 15 2022 1:38PM,234463,10,ISDIN Corporation
37,Apr 15 2022 1:33PM,234464,22,"NBTY Global, Inc."
38,Apr 15 2022 1:29PM,234462,18,"ClearSpec, LLC"
39,Apr 15 2022 1:20PM,234461,10,ISDIN Corporation
55,Apr 15 2022 12:54PM,234454,20,Alumier Labs Inc.
64,Apr 15 2022 12:47PM,234452,20,Reliable 1 Laboratories LLC
70,Apr 15 2022 12:12PM,234451,19,ACG North America LLC
71,Apr 15 2022 12:12PM,234450,18,"Blinc International, Inc."
72,Apr 15 2022 12:06PM,234449,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 15 2022 1:47PM,234465,12,Twinlab Consolidated Corporation,,,1
1,Apr 15 2022 1:38PM,234463,10,ISDIN Corporation,,36,
2,Apr 15 2022 1:33PM,234464,22,"NBTY Global, Inc.",,1,
3,Apr 15 2022 1:29PM,234462,18,"ClearSpec, LLC",,,1
4,Apr 15 2022 1:20PM,234461,10,ISDIN Corporation,,,16
5,Apr 15 2022 12:54PM,234454,20,Alumier Labs Inc.,,1,8
6,Apr 15 2022 12:47PM,234452,20,Reliable 1 Laboratories LLC,,,6
7,Apr 15 2022 12:12PM,234451,19,ACG North America LLC,,,1
8,Apr 15 2022 12:12PM,234450,18,"Blinc International, Inc.",,,1
9,Apr 15 2022 12:06PM,234449,20,"ACI Healthcare USA, Inc.",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 15 2022 1:47PM,234465,12,Twinlab Consolidated Corporation,1,,
1,Apr 15 2022 1:38PM,234463,10,ISDIN Corporation,,36,
2,Apr 15 2022 1:33PM,234464,22,"NBTY Global, Inc.",,1,
3,Apr 15 2022 1:29PM,234462,18,"ClearSpec, LLC",1,,
4,Apr 15 2022 1:20PM,234461,10,ISDIN Corporation,16,,
5,Apr 15 2022 12:54PM,234454,20,Alumier Labs Inc.,8,1,
6,Apr 15 2022 12:47PM,234452,20,Reliable 1 Laboratories LLC,6,,
7,Apr 15 2022 12:12PM,234451,19,ACG North America LLC,1,,
8,Apr 15 2022 12:12PM,234450,18,"Blinc International, Inc.",1,,
9,Apr 15 2022 12:06PM,234449,20,"ACI Healthcare USA, Inc.",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 15 2022 1:47PM,234465,12,Twinlab Consolidated Corporation,1,,
1,Apr 15 2022 1:38PM,234463,10,ISDIN Corporation,,36,
2,Apr 15 2022 1:33PM,234464,22,"NBTY Global, Inc.",,1,
3,Apr 15 2022 1:29PM,234462,18,"ClearSpec, LLC",1,,
4,Apr 15 2022 1:20PM,234461,10,ISDIN Corporation,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 15 2022 1:47PM,234465,12,Twinlab Consolidated Corporation,1.0,NaN,NaN
1,Apr 15 2022 1:38PM,234463,10,ISDIN Corporation,NaN,36.0,NaN
2,Apr 15 2022 1:33PM,234464,22,"NBTY Global, Inc.",NaN,1.0,NaN
3,Apr 15 2022 1:29PM,234462,18,"ClearSpec, LLC",1.0,NaN,NaN
4,Apr 15 2022 1:20PM,234461,10,ISDIN Corporation,16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 15 2022 1:47PM,234465,12,Twinlab Consolidated Corporation,1.0,0.0,0.0
1,Apr 15 2022 1:38PM,234463,10,ISDIN Corporation,0.0,36.0,0.0
2,Apr 15 2022 1:33PM,234464,22,"NBTY Global, Inc.",0.0,1.0,0.0
3,Apr 15 2022 1:29PM,234462,18,"ClearSpec, LLC",1.0,0.0,0.0
4,Apr 15 2022 1:20PM,234461,10,ISDIN Corporation,16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2812851,55.0,37.0,0.0
12,234465,1.0,0.0,0.0
18,3984736,118.0,0.0,0.0
19,937782,3.0,3.0,0.0
20,2344227,86.0,10.0,9.0
21,1172145,3.0,2.0,0.0
22,234464,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2812851,55.0,37.0,0.0
1,12,234465,1.0,0.0,0.0
2,18,3984736,118.0,0.0,0.0
3,19,937782,3.0,3.0,0.0
4,20,2344227,86.0,10.0,9.0
5,21,1172145,3.0,2.0,0.0
6,22,234464,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,55.0,37.0,0.0
1,12,1.0,0.0,0.0
2,18,118.0,0.0,0.0
3,19,3.0,3.0,0.0
4,20,86.0,10.0,9.0
5,21,3.0,2.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,12,Released,1.0
2,18,Released,118.0
3,19,Released,3.0
4,20,Released,86.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,18,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,9.0,0.0,0.0
Executing,37.0,0.0,0.0,3.0,10.0,2.0,1.0
Released,55.0,1.0,118.0,3.0,86.0,3.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,9.0,0.0,0.0
1,Executing,37.0,0.0,0.0,3.0,10.0,2.0,1.0
2,Released,55.0,1.0,118.0,3.0,86.0,3.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,9.0,0.0,0.0
1,Executing,37.0,0.0,0.0,3.0,10.0,2.0,1.0
2,Released,55.0,1.0,118.0,3.0,86.0,3.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()